In [2]:
# import the necessary libs
from __future__ import division 
import pandas as pd

In [3]:
df = pd.read_csv('intron_normalized_DESEQ2_READS.csv')

In [4]:
df.rename(columns={'Unnamed: 0':'GeneID'}, inplace=True)

In [5]:
df.head(1)

,GeneID,N2_CTR_1,N2_CTR_3,N2_CTR_4,N2_HS_1,N2_HS_3,N2_HS_4
0,WBGene00022277_1,2.774708,2.157659,5.264824,0.0,1.590446,5.212309


In [6]:
# get rid of the barcode so that all introns for a gene can be combined 

new_Gene_ID = []
for index,row in df.iterrows():
    a = row.GeneID
    a = a.split("_")[0]
    new_Gene_ID.append(a)

In [7]:
df['new_Gene_ID'] = new_Gene_ID

In [8]:
df.drop(['N2_CTR_1', 'N2_CTR_3', 'N2_CTR_4'], inplace=True, axis=1)

In [10]:
df.head(1)  # new_Gene_ID looks good, no barcode

,GeneID,N2_HS_1,N2_HS_3,N2_HS_4,new_Gene_ID
0,WBGene00022277_1,0.0,1.590446,5.212309,WBGene00022277


In [12]:
### sum introns of same gene

df = df.groupby('new_Gene_ID').sum()

In [13]:
df['intron_HS_total'] = df['N2_HS_1'] + df['N2_HS_3'] + df['N2_HS_4']


In [14]:
df['intron_mean'] = df[['N2_HS_1','N2_HS_3','N2_HS_4']].mean(axis=1)

In [15]:
df.drop(['N2_HS_1','N2_HS_3','N2_HS_4'], inplace=True, axis=1)

In [16]:
df.reset_index(inplace=True)

In [17]:
# these are the reads generated from deseq2 for the exons of all genes

df_exon = pd.read_csv('normalized_DESEQ2_READS.csv')

In [30]:
df_exon.rename(columns={'Unnamed: 0':'new_Gene_ID'}, inplace=True)

In [31]:
df_exon.drop(['N2_CTR_1','N2_CTR_3','N2_CTR_4'], axis=1, inplace=True)

KeyError: "['N2_CTR_1' 'N2_CTR_3' 'N2_CTR_4'] not found in axis"

In [32]:
df_exon['exon_HS_total'] = df_exon['N2_HS_1'] + df_exon['N2_HS_3'] + df_exon['N2_HS_4']

In [33]:
df_exon['exon_mean'] = df_exon[['N2_HS_1', 'N2_HS_3', 'N2_HS_4']].mean(axis=1)
df_exon.head(1)

,new_Gene_ID,N2_HS_1,N2_HS_3,N2_HS_4,exon_HS_total,exon_mean
0,WBGene00015153,238.525939,48.545685,66.075503,353.147127,117.715709


In [34]:
merged = pd.merge(df, df_exon, how='inner', on='new_Gene_ID')

In [35]:
## Calculate Intron retention ratio by dividing intron_total /exon_tota 

merged['ratio'] = merged['intron_HS_total']/merged['exon_HS_total']
merged['ave_ratio'] = merged['intron_mean']/merged['exon_mean']

In [36]:
#merged=merged[merged['exon_mean']>0]

In [25]:
merged.rename(columns={'new_Gene_ID':'WB_ID'},inplace=True)

In [26]:
merged_for_saving=merged[['WB_ID', 'ratio']]

In [27]:
merged_for_saving.to_csv('intron_retention_score.csv')


In [39]:
###############################################################
! grep WBGene00000023 intron_retention_score.csv

18,WBGene00000023,0.08827634072928496


 # End Notebook
 # below code can be used to merge to up genes 
 

In [29]:
up = pd.read_csv('unfiltered_list_of_up.txt')

IOError: [Errno 2] File unfiltered_list_of_up.txt does not exist: 'unfiltered_list_of_up.txt'

In [ ]:
up.drop('Unnamed: 0', inplace=True, axis=1)

In [61]:
merged_up=pd.merge(up,merged,how='inner',on='WB_ID')

In [62]:
len(merged_up[merged_up.ratio>1])

19

In [63]:
ir=merged_up[['WB_ID','ratio']]
ir=ir.rename(columns={'WB_ID':'Geneid','ratio':'IR_score'})

In [64]:
ir[ir.Geneid.str.contains("WBGene00002015")]

,Geneid,IR_score
173,WBGene00002015,0.388706


In [65]:
len(ir)

1105

In [66]:
! pwd

/home/wschrein/69_final_intron_filering
